In [ ]:
import cantera as ct
import cantera.ck2cti as ck2cti
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook

In [ ]:
# command line equivalent:
# python -m cantera.ck2cti --input=mech.txt --thermo=thermo.txt --transport=tran.txt --permissive --out=mech.cti
parser = ck2cti.Parser()
#parser.convertMech('mech.txt', 'thermo.txt', 'tran.txt', outName='mech.cti')
#parser.convertMech('mech_fixed.txt', 'thermo.txt', 'tran.txt', outName='mech.cti')
#parser.convertMech('mech_fixed.txt', 'thermo_fix.txt', 'tran.txt', outName='mech.cti')
parser.convertMech('mech_fixed.txt', 'thermo_fixed.txt', 'tran.txt', permissive=True, outName='mech.cti')

In [ ]:
gas = ct.Solution('mech.cti')

In [ ]:
sp = gas.species('H2CNO')
T = np.linspace(600, 2000, 200)
f,ax = plt.subplots(1,3, figsize=(8,3.5))

def plot_thermo(thermo):
    h = [thermo.h(tt)/(ct.gas_constant * tt) for tt in T]
    cp = [thermo.cp(tt)/ct.gas_constant for tt in T]
    s = [thermo.s(tt)/ct.gas_constant for tt in T]
    ax[0].plot(T,cp)
    ax[0].set_title('$c_p/R$')
    ax[1].plot(T,h)
    ax[1].set_title('$h/RT$')
    ax[2].plot(T,s)
    ax[2].set_title('$s/R$')
    f.tight_layout()
plot_thermo(sp.thermo)

In [ ]:
c0 = sp.thermo.coeffs
c0

In [ ]:
c0[0] = 1200
test = ct.NasaPoly2(sp.thermo.min_temp, sp.thermo.max_temp, sp.thermo.reference_pressure, c0)
plot_thermo(test)

In [ ]:
c0[0] = 1000
test = ct.NasaPoly2(sp.thermo.min_temp, sp.thermo.max_temp, sp.thermo.reference_pressure, c0)
plot_thermo(test)